<a href="https://colab.research.google.com/github/natexjake/OllopaCorporation/blob/main/OOP_Facedetection_Media_Pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
from google.colab.patches import cv2_imshow
from google.colab import files
from google.colab import drive
import PIL
import io
import os
import sys
import cv2
import html
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [14]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
package_path = '/content/packages'
sys.path.insert(0,package_path)

In [16]:
import mediapipe as mp

In [20]:
def browse_photos():
    print('\n')
    print("Browsing photos...")
    uploaded = files.upload()

    # Iterate over the uploaded files
    for filename, file_content in uploaded.items():
        # Decode the file content as an image
        nparr = np.frombuffer(file_content, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # Display the uploaded image
        # cv2_imshow(img)

        mp_face_mesh = mp.solutions.face_mesh
        face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
        results = face_mesh.process(img[:,:,::-1])
        landmarks = results.multi_face_landmarks[0]

        face_oval = mp_face_mesh.FACEMESH_FACE_OVAL

        df = pd.DataFrame(list(face_oval), columns=["p1", "p2"])

        routes_idx = []

        p1 = df.iloc[0]["p1"]
        p2 = df.iloc[0]["p2"]

        for i in range(0, df.shape[0]):
            obj = df[df["p1"] == p2]
            p1 = obj["p1"].values[0]
            p2 = obj["p2"].values[0]

            current_route = []
            current_route.append(p1)
            current_route.append(p2)
            routes_idx.append(current_route)

        routes = []

        for source_idx, target_idx in routes_idx:
            source = landmarks.landmark[source_idx]
            target = landmarks.landmark[target_idx]

            relative_source = (int(source.x * img.shape[1]), int(source.y * img.shape[0]))
            relative_target = (int(target.x * img.shape[1]), int(target.y * img.shape[0]))

            routes.append(relative_source)
            routes.append(relative_target)

        mask = np.zeros((img.shape[0], img.shape[1]))
        mask = cv2.fillConvexPoly(mask, np.array(routes), 1)
        mask = mask.astype(bool)

        out = np.zeros_like(img)
        out[mask] = img[mask]

        # Display the manipulated image
        cv2_imshow(out)

In [21]:
def use_webcam():
    print('\n')
    print("Using webcam...")

    # JavaScript code to capture image from webcam
    js_code = """
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Wait for Capture to be clicked.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();

        const dataUrl = canvas.toDataURL('image/jpeg', quality);

        // Return the captured image data URL
        return dataUrl;
    }
    """

    # Execute the JavaScript code
    display(Javascript(js_code))

    # Call the JavaScript function to capture the image
    data = eval_js('takePhoto(0.8)')

    # Convert the captured image data URL to an OpenCV image
    img = js_to_image(data)

    # Save the captured image
    filename = 'captured_image.jpg'
    cv2.imwrite(filename, img)
    print('Saved to {}'.format(filename))

    # Display the captured image
    cv2_imshow(img)

    # You can continue processing the captured image as needed

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

# initialize the Mediapipe face detection model
mp_face_detection = mp.solutions.face_detection.FaceDetection()

def detect_faces(image):
    # convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # detect faces using Mediapipe face detection
    results = mp_face_detection.process(image_rgb)

    # extract face bounding boxes
    faces = []
    if results.detections:
        for detection in results.detections:
            bounding_box = detection.location_data.relative_bounding_box
            image_height, image_width, _ = image.shape
            x = int(bounding_box.xmin * image_width)
            y = int(bounding_box.ymin * image_height)
            w = int(bounding_box.width * image_width)
            h = int(bounding_box.height * image_height)
            faces.append((x, y, w, h))

    return faces

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # # get OpenCV format image
  img = js_to_image(data)
  # # grayscale img
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  # # get face bounding box coordinates using Haar Cascade
  # faces = face_cascade.detectMultiScale(gray)
  # # draw face bounding box on image
  # for (x,y,w,h) in faces:
  #     img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
  # # save image
  cv2.imwrite(filename, img)

  return filename

  #Check if the choice is "2" (capture image from the camera)
  if choice == "2":
    try:
      filename = take_photo('photo.jpg')
      print('Saved to {}'.format(filename))

      # Show the image which was just taken.
      display(Image(filename))
    except Exception as err:
      # Errors will be thrown if the user does not have a webcam or if they do not
      # grant the page permission to access it.
      print(str(err))

In [ ]:
def main():
    print("Face Detection Program")
    print("----------------------")
    print("1. Upload an image")
    print("2. Capture an image from the camera")
    choice = input("Enter your choice (1 or 2): ")

    if choice == "1":
        browse_photos()
    elif choice == "2":
        use_webcam()
    else:
        print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()
